In [ ]:
import pandas as pd

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics
import numpy as np

In [ ]:
from multiprocessing import Pool

In [ ]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

In [ ]:
flash_names = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*dash\\input\\*', '_first_edit_cs.csv')

In [ ]:
gn = global_names = preproc_tool.GlobalNames()

In [ ]:
def calculated_regime_time(this_file, regime_column = gn.i_a_motor_a,   return_all = False):
    last_time = this_file.index[0]

    work_status = []
    if this_file[regime_column][0] > 0:
        work_status.append(1)
    else:
        work_status.append(0)

    work_timedelta = []
    stop_timedelta = []
    regime_bounds = []
    work_bounds = []
    stop_bounds = []


    for i in range(1, this_file.shape[0]):
        this_time = this_file.index[i]
        this_value = this_file[gn.i_a_motor_a][i]
        if this_value > 0:
            work_status.append(1)
        else:
            work_status.append(0)
        #print(this_value)
        time_delta = this_time - last_time
        #time_delta = this_file.index[i-1] - last_time    
        if work_status[-1] != work_status[-2]:
            
            #print('Переключение')
            if work_status[-1] ==1:
                #print('Включение')
                stop_timedelta.append(time_delta.total_seconds()/60)
                stop_bounds.append([last_time, this_time])
            else:
                #print('Выключение')
                work_timedelta.append(time_delta.total_seconds()/60)
                work_bounds.append([last_time, this_time])
                
            last_time = this_time
            
            
    work_time_median = np.median(work_timedelta)
    stop_timedelta_median = np.median(stop_timedelta)
    if not return_all:
        return work_time_median, stop_timedelta_median
    else:
        return work_status, work_timedelta, stop_timedelta, work_bounds, stop_bounds

In [ ]:
def create_useful_inf(cs_file_name):
    info_dict = {}
    this_file = pd.read_csv(cs_file_name, index_col = [0], parse_dates = True)
    print(cs_file_name.split('\\')[-1].replace('_first_edit_cs.csv',''))
    this_file = preproc_tool.rename_columns_by_dict(this_file)
    info_dict['Начало записи данных'] = [this_file.index[0]]
    info_dict['Конец записи данных'] = [this_file.index[-1]]
    info_dict['Продолжительность записи данных'] = [this_file.index[-1] - this_file.index[0]]
    info_dict['Общее число записей'] =   [this_file.shape[0]]
    info_dict['Средняя дискретность, секунды'] =   [((this_file.index[-1] - this_file.index[0])/this_file.shape[0]).seconds]
    info_dict['Медианное время работы, мин'], info_dict['Медианное время простаивания, мин'] = calculated_regime_time(this_file)
    if 'F Турб.вращ.,Гц' in this_file.columns:
        info_dict['Медианная частота турбинного вращения'] =   [this_file[this_file['F Турб.вращ.,Гц']>0]['F Турб.вращ.,Гц'].median()]
        info_dict['Максимальная частота турбинного вращения'] =   [this_file['F Турб.вращ.,Гц'].max()]
    else:
        info_dict['Медианная частота турбинного вращения'] =   [-1]
        info_dict['Максимальная частота турбинного вращения'] =   [-1]
    if gn.freq_hz in this_file.columns:
        this_file = this_file[this_file[gn.freq_hz]>0]
    else:
        this_file = this_file[this_file[gn.i_a_motor_a]>0]
    
    
    info_dict['Медианное значение загрузки (Р)'] =   [ this_file[gn.motor_load_perc].median()]
    info_dict['Максимальное значение загрузки (Р)'] =   [ this_file[gn.motor_load_perc].max()]
    
    info_dict['Медианное значение тока (Р)'] =   [this_file[gn.i_a_motor_a].median()]
    info_dict['Максимальное значение тока (Р)'] =   [this_file[gn.i_a_motor_a].max()]

    info_dict['Медианное значение мощности (Р)'] =   [this_file[gn.active_power_kwt].median()]
    info_dict['Максимальное значение мощности (Р)'] =   [this_file[gn.active_power_kwt].max()]
    
    if gn.freq_hz in this_file.columns:
        info_dict['Медианное значение частоты (Р)'] =   [this_file[gn.freq_hz].median()]
        info_dict['Максимальное значение частоты (Р)'] =   [this_file[gn.freq_hz].max()]
    else:
        info_dict['Медианное значение частоты (Р)'] =   [-1]
        info_dict['Максимальное значение частоты (Р)'] =   [-1]        
    
    if gn.p_intake_atm in this_file.columns:
        info_dict['Медианное значение давления на приеме (Р)'] =   [this_file[gn.p_intake_atm].median()]
        info_dict['Минимальное значение давления на приеме (Р)'] =   [this_file[gn.p_intake_atm].min()]
    else:
        info_dict['Медианное значение давления на приеме (Р)'] =   [-1]
        info_dict['Минимальное значение давления на приеме (Р)'] =   [-1]
    
    if gn.t_motor_c in this_file.columns:
        info_dict['Медианное значение температуры двигателя (Р)'] =   [this_file[gn.t_motor_c].median()]
        info_dict['Максимальное значение температуры двигателя (Р)'] =   [this_file[gn.t_motor_c].max()]
    else:
        info_dict['Медианное значение температуры двигателя (Р)'] =   [-1]
        info_dict['Максимальное значение температуры двигателя (Р)'] = [-1]
    
    info_dict['Медианное значение коэффициента мощности (Р)'] =   [this_file[gn.cos_phi_d].median()]
    info_dict['Минимальное значение коэффициента мощности (Р)'] =   [this_file[gn.cos_phi_d].min()]

    


    
    result_df = pd.DataFrame(info_dict)
    result_df.index = [cs_file_name.split('\\')[-1].replace('_first_edit_cs.csv','')]
    result_df.index.name = 'Скважина'
    return result_df

In [ ]:
overall_result = None
for i in flash_names:
    this_df = create_useful_inf(i)
    try:
        overall_result = overall_result.append(this_df)
    except:
        overall_result = this_df.copy()

In [ ]:
overall_result.to_excel('ПКВ.xlsx')

In [ ]:
this_name = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*dash\\input\\*', '898_first_edit_cs.csv')

In [ ]:
this_file = pd.read_csv(this_name[0], index_col = [0], parse_dates = True)


In [ ]:
def prepare_data(this_file):
    this_file = preproc_tool.rename_columns_by_dict(this_file)
    
    if this_file[gn.i_a_motor_a].min() == 0.1:
        this_file[gn.i_a_motor_a] = this_file[gn.i_a_motor_a].replace(0.1, 0)
        
        
    work_status, work_timedelta, stop_timedelta, work_bounds, stop_bounds  = calculated_regime_time(this_file, 
                                                                         regime_column = gn.i_a_motor_a, return_all = True)
    this_file['work_status'] = work_status
    return this_file

def total_seconds(timedelta):
    """Convert timedeltas to seconds
    In Python, time differences can take many formats. This function can take
    timedeltas in any format and return the corresponding number of seconds, as
    a float.
    Beware! Representing timedeltas as floats is not as precise as representing
    them as a timedelta object in datetime, numpy, or pandas.
    Parameters
    ----------
    timedelta : various
        Time delta from python's datetime library or from numpy or pandas. If
        it is from numpy, it can be an ndarray with dtype datetime64. If it is
        from pandas, it can also be a Series of datetimes. However, this
        function cannot operate on entire pandas DataFrames. To convert a
        DataFrame, do df.apply(to_seconds)
    Returns
    -------
    seconds : various
        Returns the total seconds in the input timedelta object(s) as float.
        If the input is a numpy ndarray or pandas Series, the output is the
        same, but with a float datatype.
    """
    try:
        seconds = timedelta.total_seconds()
    except AttributeError:  # no method total_seconds
        one_second = np.timedelta64(1000000000, 'ns')
        # use nanoseconds to get highest possible precision in output
        seconds = timedelta / one_second
    return seconds

def get_true_median_value(df, column, except_zero = True):
    this_df = df.copy()
    #this_df = this_df.resample('1s').mean()
    this_df = this_df.interpolate('linear')
    if except_zero:    
        this_df = this_df[this_df[column]>0]
    median_value = this_df[column].median()
    if df[column].median() > median_value:
        median_value = df[column].median()
    
    
    top_values_len = this_file[this_file[column] > this_file[column].max()*0.9].shape[0]
    _, work_timedelta, _,_,_ = calculated_regime_time(df, regime_column = column, return_all = True)
    if len(work_timedelta) * 2 < top_values_len:
        median_value = this_file[this_file[column] > this_file[column].max()*0.9][column].median()
    print(median_value)
    return median_value


def find_stucks(this_file):
    median_value = get_true_median_value(this_file, gn.i_a_motor_a)
    stuck_times = this_file[this_file[gn.i_a_motor_a] > median_value * 2]
    stucks = []
    if stuck_times.shape[0] > 0:
        left_boundary = stuck_times.index[0]
        this_stuck_interval = None
        for i in range(stuck_times.shape[0]-1):
            right_boundary = stuck_times.index[i+1]
            stuck_times_rows = stuck_times[(stuck_times.index >= left_boundary) & (stuck_times.index <= right_boundary)].shape[0]
            this_file_rows = this_file[(this_file.index >= left_boundary) & (this_file.index <= right_boundary)].shape[0]
            if stuck_times_rows == this_file_rows:
                this_stuck_interval = [left_boundary, right_boundary]        
            else:
                stucks.append(this_stuck_interval)
                left_boundary = stuck_times.index[i+1]
            if i == stuck_times.shape[0]-2:
                stucks.append(this_stuck_interval)
    return stucks



def create_shapes_to_plotly(borders):
    shapes = []
    for i in borders:
        this_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=i[0],
            y0=0,
            x1=i[1],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.9,
            layer="below",
            line_width=1,
            line_color="LightSalmon"
        )
        shapes.append(this_shape)
    return shapes
     
def find_stucks(df):
    this_file = df.copy()
    this_file['Ток фазы А дельта'] = [0] + list(this_file[gn.i_a_motor_a].values[1::] - this_file[gn.i_a_motor_a].values[0:-1])

    time_series = this_file['Ток фазы А дельта']
    time_series = time_series[(time_series <  np.inf) &  (time_series > -np.inf)]
    work_status, work_timedelta, stop_timedelta, work_bounds, stop_bounds = calculated_regime_time(this_file, 
                                                                             regime_column = gn.i_a_motor_a, return_all = True)
    stucks = []

    not_all_stucks_saved = True

    value = time_series.min()
    while not_all_stucks_saved:
        index = time_series.where(time_series == value).dropna().index[0]

        for number, i in enumerate(work_bounds):
            if index>= i[0] and index<=i[1]:
                #print(i)
                stuck_work_bounds = i
                normal_work_bounds = work_bounds[number-1]
                stuck_work_series = time_series[(time_series.index>=stuck_work_bounds[0]) & (time_series.index<stuck_work_bounds[1])]
                normal_work_series = time_series[(time_series.index>=normal_work_bounds[0]) & (time_series.index<normal_work_bounds[1])]
                stuck_work_timedelta = stuck_work_series.index[-1] - stuck_work_series.index[0]
                normal_work_timedelta = normal_work_series.index[-1] - normal_work_series.index[0]
                if total_seconds(normal_work_timedelta) !=0:
                    if total_seconds(stuck_work_timedelta) / total_seconds(normal_work_timedelta) < 0.25:
                        stucks.append(stuck_work_bounds)
                        time_series = time_series[(time_series.index < i[0]) | (time_series.index > i[1])]
                        new_value = time_series.min()
        try:
            if new_value != value:
                value = new_value
            else:
                not_all_stucks_saved = False
            
        except:
            not_all_stucks_saved = False
            
    return stucks

In [ ]:
this_file = prepare_data(this_file)

In [ ]:
stucks = find_stucks(this_file)

In [ ]:
stucks

In [ ]:
this_file['index дельта'] = [0] + list(this_file.index.values[1::] - this_file.index.values[0:-1])
this_file['Ток фазы А дельта'] = [0] + list(this_file[gn.i_a_motor_a].values[1::] - this_file[gn.i_a_motor_a].values[0:-1])

this_file['Ток фазы А производная'] = [0] + list(this_file['Ток фазы А дельта'][1::] /
                                                 this_file['index дельта'][1::].apply(total_seconds))

In [ ]:
stucks = find_stucks(this_file)
stucks

In [ ]:
overall_result = None
for i in flash_names:
    this_file = pd.read_csv(i, index_col = [0], parse_dates = True)
    this_file = prepare_data(this_file)
    stucks = find_stucks(this_file)
    print(i)
    print(len(stucks))

In [ ]:
this_name = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*dash\\initial_files\\*', '.xml')

In [ ]:
import xml.etree.ElementTree as ET
root = ET.parse(this_name[0]).getroot()

In [ ]:
root.tag

In [ ]:
for child in root:
    print(child.tag, child.attrib)

In [ ]:
root[0][0][3].text

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

def iter_docs(author):
    author_attr = author.attrib
    for doc in author.iter('document'):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)
        doc_dict['data'] = doc.text
        yield doc_dict


etree = ET.parse(this_name[0]) #create an ElementTree object 
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

In [ ]:
from bs4 import BeautifulSoup
 
def read_excel_xml(path):
    file = open(path).read()
    soup = BeautifulSoup(file,'xml')
    workbook = []
    for sheet in soup.findAll('Worksheet'): 
        sheet_as_list = []
        for row in sheet.findAll('Row'):
            row_as_list = []
            for cell in row.findAll('Cell'):
                row_as_list.append(cell.Data.text)
            sheet_as_list.append(row_as_list)
        workbook.append(sheet_as_list)
    return workbook

In [ ]:
handler = open(this_name[0]).read()
soup = BeautifulSoup(handler, 'lxml')

tables = []
for table in soup.findAll('ss:table'):
    rows = []
    for row in table.findAll('ss:row'):
        cells = []
        for cell in row.findAll('ss:cell'):
            cells.append(cell.find('ss:data').get_text())
        rows.append(cells)
    tables.append(rows)

# Parsed sheets
tables

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=this_file[gn.i_a_motor_a].index,
    y=this_file[gn.i_a_motor_a],
    mode="lines",
    name="temperature"
))

# Add shape regions
fig.update_layout(
    shapes= create_shapes_to_plotly(stucks)
)

fig.show()